In [63]:
import pandas as pd
df = pd.read_csv('./datas/문항_50000_Questions_Data.csv')
df2 = pd.read_csv('./datas/선제조건_사전평가.csv')
# df2 = pd.read_csv('./datas/선제조건_구술_사전평가.csv')

In [ ]:
df.head()

In [ ]:
df2.head()

In [64]:
import pandas as pd
import numpy as np

def generate_exam_questions(df, df2):
    def sample_matching_rows(df, df2_row, used_item_codes):
        conditions = (
            (df['평가종류'] == df2_row['평가종류']) &
            (df['출제단계'] == df2_row['출제단계']) &
            (df['문항유형'] == df2_row['문항유형']) &
            (df['평가영역'] == df2_row['평가영역']) &
            (~df['문항코드'].isin(used_item_codes))
        )
        return df[conditions].sample(n=1) if not df[conditions].empty else None

    df3_rows = []
    used_item_codes = set()

    for _, df2_row in df2.iterrows():
        matched_df_row = sample_matching_rows(df, df2_row, used_item_codes)
        
        if matched_df_row is not None:
            item_code = matched_df_row['문항코드'].values[0]
            used_item_codes.add(item_code)
        else:
            item_code = np.nan

        df3_rows.append({
            '문제번호': df2_row['문제번호'],
            '문항코드': item_code,
            '평가종류': df2_row['평가종류'] if matched_df_row is None else matched_df_row['평가종류'].values[0],
            '배점': df2_row['배점'],
            '출제단계': df2_row['출제단계'] if matched_df_row is None else matched_df_row['출제단계'].values[0],
            '문항유형': df2_row['문항유형'] if matched_df_row is None else matched_df_row['문항유형'].values[0],
            '난이도': df2_row['난이도'],
            '평가영역': df2_row['평가영역'] if matched_df_row is None else matched_df_row['평가영역'].values[0],
            '평가항목': df2_row['평가항목']
        })

    df3 = pd.DataFrame(df3_rows)

    # 매칭되지 않은 행의 수 출력
    unmatched_rows = df3['문항코드'].isna().sum()
    if unmatched_rows > 0:
        print(f"주의: {unmatched_rows}개의 행에 대해 매칭되는 문항을 찾지 못했습니다.")
    
    return df3


In [65]:
exam1 = generate_exam_questions(df, df2)
exam2 = generate_exam_questions(df, df2)
exam3 = generate_exam_questions(df, df2)

In [66]:
# exam1, exam2, exam3를 csv 파일로 저장
exam1.to_csv('exam1.csv', index=False)
exam2.to_csv('exam2.csv', index=False)
exam3.to_csv('exam3.csv', index=False)